In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mapping_helper_functions import convert_latitude_to_webmercator, convert_longitude_to_webmercator
from rossmo_et_ridgway import Rossmo

In [2]:
# body_locations_path = '../resources/Ridgway/body_locations.csv'
ridgway_locations_path = '../resources/Ridgway/ridgway_locations.csv'

# df_victims = pd.read_csv(body_locations_path)
df_ridgway = pd.read_csv(ridgway_locations_path)

In [3]:
df_ridgway['coordinates'] = list(zip(df_ridgway['Y'], df_ridgway['X']))
df_ridgway['latitude_webmercator'] = convert_latitude_to_webmercator(df_ridgway['Y'])
df_ridgway['longitude_webmercator'] = convert_longitude_to_webmercator(df_ridgway['X'])

## TODO: 

- have the Rossmo class do the following:
  - convert coordinates column from df to list, so user doesn't have to
  - return the dataframe 
  - add the score_normalized column
  - set anything below q1 to score_normalized = 0 by default
- decide whether class should handle plotting as well

In [15]:
disappearances_path = '../resources/Ridgway/disappearances.csv'
df_disappearances = pd.read_csv(disappearances_path)

df_disappearances['coordinates'] = list(zip(df_disappearances['Y'], df_disappearances['X']))

In [16]:
df_disappearances.sample(5)

,X,Y,Name,description,coordinates
39,-122.323976,47.598712,Shirley Marie Sherrill,NaN,"(47.5987122, -122.3239762)"
8,-122.294507,47.435612,Constance Elizabeth Naon,NaN,"(47.435612, -122.2945071)"
7,-122.326415,47.592023,Colleen Renee Brockman,NaN,"(47.5920232, -122.3264155)"
32,-122.275231,47.536055,Pammy Annette Avent,NaN,"(47.5360554, -122.2752313)"
28,-122.285058,47.557850,Mary Exzetta West,NaN,"(47.5578503, -122.2850579)"


In [52]:
R = Rossmo(
    df_disappearances['coordinates'].to_list(), 
    accuracy=200
)

In [53]:
rossmo_results = R.rossmo_results

In [54]:
df_rossmo_results = pd.DataFrame({'coordinates': rossmo_results.keys(), 'score': rossmo_results.values()})

In [55]:
df_rossmo_results['score_normalized'] = (
    (df_rossmo_results['score'] - df_rossmo_results['score'].min()) / (df_rossmo_results['score'].max() - df_rossmo_results['score'].min()) 
) 
df_rossmo_results.sample(5)

,coordinates,score,score_normalized
6967,"(47.02313894901232, -122.89695427668947)",43.626923,0.028514
34949,"(48.0101198213524, -122.75551897521714)",46.249262,0.038385
2306,"(46.86099209141359, -122.41764575503326)",52.129014,0.060517
23674,"(47.61532747241637, -122.16620521908246)",91.602598,0.209099
10329,"(47.1429866263679, -122.5983686402479)",56.347651,0.076396


In [56]:
# set normalized score for score values below q1 to 0.0
# q1 = stats['25%']
# df_rossmo_results.loc[df_rossmo_results['score'] <= q1, ['score_normalized']] = 0.0

In [57]:
df_rossmo_results.loc[df_rossmo_results['score_normalized'] > 0].sort_values(by='score_normalized', ascending=False).head(5)

,coordinates,score,score_normalized
19689,"(47.474330204939214, -122.2840679703094)",301.719766,1.000000
19889,"(47.48138006831307, -122.2840679703094)",226.507920,0.716895
19489,"(47.467280341565356, -122.2840679703094)",219.814583,0.691701
18490,"(47.432031024696066, -122.29192548705785)",215.140767,0.674108
18690,"(47.439080888069924, -122.29192548705785)",212.798652,0.665292


In [58]:
df_rossmo_results[['latitude', 'longitude']] = pd.DataFrame(df_rossmo_results['coordinates'].tolist(), index=df_rossmo_results.index)

# convert latitude and longitude to web mercator for plotting
df_rossmo_results['latitude_webmercator'] = convert_latitude_to_webmercator(df_rossmo_results['latitude'])
df_rossmo_results['longitude_webmercator'] = convert_longitude_to_webmercator(df_rossmo_results['longitude'])

In [67]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON_RETINA, get_provider
from bokeh.models import HoverTool

# show map in notebook
output_notebook()

tile_provider = get_provider(CARTODBPOSITRON_RETINA)

x_range = df_rossmo_results['longitude_webmercator'].min(), df_rossmo_results['longitude_webmercator'].max()  # lon
y_range = df_rossmo_results['latitude_webmercator'].min(), df_rossmo_results['latitude_webmercator'].max() # lat 

# range bounds supplied in web mercator coordinates
p = figure(
    x_range=x_range, 
    y_range=y_range,
    x_axis_type='mercator', 
    y_axis_type='mercator',
    lod_threshold=None
)
p.add_tile(tile_provider)

d = np.fliplr(
    df_rossmo_results['score_normalized']\
    .to_numpy()\
    .reshape((R.accuracy, R.accuracy))
)

# plot the heatmap
p.image(
    image=[d], 
    x=x_range[0],
    y=y_range[0],
    dw=abs(x_range[1] - x_range[0]), 
    dh=abs(y_range[1] - y_range[0]), 
    palette='Spectral10', 
    alpha=0.65
)

p.circle(
    x=df_ridgway['longitude_webmercator'],
    y=df_ridgway['latitude_webmercator']
)

show(p)

Loading BokehJS ...

In [60]:
top_10 = (
    df_rossmo_results.loc[df_rossmo_results['score_normalized'] > 0]\
    .sort_values('score_normalized', ascending=False)[:10]\
    .reset_index()
)
top_10

,index,coordinates,score,score_normalized,latitude,longitude,latitude_webmercator,longitude_webmercator
0,19689,"(47.474330204939214, -122.2840679703094)",301.719766,1.000000,47.474330,-122.284068,6.019843e+06,-1.361260e+07
1,19889,"(47.48138006831307, -122.2840679703094)",226.507920,0.716895,47.481380,-122.284068,6.021005e+06,-1.361260e+07
2,19489,"(47.467280341565356, -122.2840679703094)",219.814583,0.691701,47.467280,-122.284068,6.018682e+06,-1.361260e+07
3,18490,"(47.432031024696066, -122.29192548705785)",215.140767,0.674108,47.432031,-122.291925,6.012880e+06,-1.361347e+07
4,18690,"(47.439080888069924, -122.29192548705785)",212.798652,0.665292,47.439081,-122.291925,6.014040e+06,-1.361347e+07
5,19690,"(47.474330204939214, -122.29192548705785)",210.991185,0.658489,47.474330,-122.291925,6.019843e+06,-1.361347e+07
6,18491,"(47.432031024696066, -122.29978300380633)",207.489921,0.645310,47.432031,-122.299783,6.012880e+06,-1.361435e+07
7,18691,"(47.439080888069924, -122.29978300380633)",205.269814,0.636953,47.439081,-122.299783,6.014040e+06,-1.361435e+07
8,18291,"(47.42498116132221, -122.29978300380633)",202.942780,0.628194,47.424981,-122.299783,6.011720e+06,-1.361435e+07
9,18290,"(47.42498116132221, -122.29192548705785)",202.264480,0.625641,47.424981,-122.291925,6.011720e+06,-1.361347e+07
